In [ ]:
pip install gluonts


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.2 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/time-series-foundation-models/lag-llama/

Cloning into 'lag-llama'...
remote: Enumerating objects: 328, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 328 (delta 113), reused 107 (delta 84), pack-reused 164
Receiving objects: 100% (328/328), 234.57 KiB | 1.67 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [ ]:
%cd /content/lag-llama


/content/lag-llama


In [ ]:
!pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 48.7 MB/s eta 0:00:00


In [ ]:
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/lag-llama

lag-llama.ckpt: 100% 29.5M/29.5M [00:00<00:00, 47.5MB/s]
Download complete. Moving file to /content/lag-llama/lag-llama.ckpt
/content/lag-llama/lag-llama.ckpt


In [ ]:
import pandas as pd
import torch
from gluonts.dataset.pandas import PandasDataset

from sklearn.model_selection import train_test_split
from itertools import islice
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from tqdm.autonotebook import tqdm
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import make_evaluation_predictions, Evaluator

import pandas as pd
from lag_llama.gluon.estimator import LagLlamaEstimator

In [ ]:
file_path = 'dataset.xlsx'
data = pd.read_excel(file_path)

# this will give u an error if you don't put the dataset inside the lag-llama folder. Be mindful abt this

In [ ]:
df = data.copy()
df['ds'] = pd.to_datetime(df['ds'])
df.set_index('ds', inplace=True)
area_columns = [col for col in df.columns if 'AREA' in col]
df = df[area_columns]

In [ ]:
train_df, temp_df = train_test_split(df, test_size=0.3, shuffle=False)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=False)

train_data = ListDataset(
    [{"start": train_df.index[0], "target": train_df[column].values} for column in train_df.columns],
    freq="H"
)
validation_data = ListDataset(
    [{"start": validation_df.index[0], "target": validation_df[column].values} for column in validation_df.columns],
    freq="H"
)
test_data = ListDataset(
    [{"start": test_df.index[0], "target": test_df[column].values} for column in test_df.columns],
    freq="H"
)

In [ ]:
def get_lag_llama_predictions(dataset, prediction_length, context_length=32, num_samples=20, device="cuda", batch_size=64, nonnegative_pred_samples=True):
    ckpt = torch.load("lag-llama.ckpt", map_location=device)
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        nonnegative_pred_samples=nonnegative_pred_samples,
        rope_scaling={
            "type": "linear",
            "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        },
        batch_size=batch_size,
        num_parallel_samples=num_samples,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(tqdm(forecast_it, total=len(dataset), desc="Forecasting batches"))
    tss = list(tqdm(ts_it, total=len(dataset), desc="Ground truth"))

    return forecasts, tss


In [ ]:
# Define parameters
prediction_length = 24
context_length = prediction_length * 3
num_samples = 20
device = "cuda"

# Fine-tune the model
ckpt = torch.load("lag-llama.ckpt", map_location=device)
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

estimator = LagLlamaEstimator(
    ckpt_path="lag-llama.ckpt",
    prediction_length=prediction_length,
    context_length=context_length,
    input_size=estimator_args["input_size"],
    n_layer=estimator_args["n_layer"],
    n_embd_per_head=estimator_args["n_embd_per_head"],
    n_head=estimator_args["n_head"],
    scaling=estimator_args["scaling"],
    time_feat=estimator_args["time_feat"],
    nonnegative_pred_samples=True,
    batch_size=64,
    num_parallel_samples=num_samples,
    trainer_kwargs={"max_epochs": 50},
)

predictor = estimator.train(train_data, cache_data=True, shuffle_buffer_length=1000)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:

forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data,
    predictor=predictor,
    num_samples=num_samples
)

forecasts = list(tqdm(forecast_it, total=len(test_data), desc="Forecasting batches"))
tss = list(tqdm(ts_it, total=len(test_data), desc="Ground truth"))



In [ ]:

plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)
    plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
    forecast.plot(color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()



In [ ]:
# Evaluate metrics
evaluator = Evaluator()
agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
print(agg_metrics)


In [ ]:
# Define parameters
prediction_length = 24
context_length = prediction_length * 3
num_samples = 20
device = "cuda"

# Fine-tune the model
ckpt = torch.load("lag-llama.ckpt", map_location=device)
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

estimator = LagLlamaEstimator(
    ckpt_path="lag-llama.ckpt",
    prediction_length=prediction_length,
    context_length=context_length,
    input_size=estimator_args["input_size"],
    n_layer=estimator_args["n_layer"],
    n_embd_per_head=estimator_args["n_embd_per_head"],
    n_head=estimator_args["n_head"],
    scaling=estimator_args["scaling"],
    time_feat=estimator_args["time_feat"],
    nonnegative_pred_samples=True,
    batch_size=32,  # Reduced batch size for better learning
    num_parallel_samples=num_samples,
    trainer_kwargs={"max_epochs": 100},  # Increased epochs
)

predictor = estimator.train(train_data, cache_data=True, shuffle_buffer_length=1000)


In [ ]:
# Evaluate the model
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data,
    predictor=predictor,
    num_samples=num_samples
)

forecasts = list(tqdm(forecast_it, total=len(test_data), desc="Forecasting batches"))
tss = list(tqdm(ts_it, total=len(test_data), desc="Ground truth"))

# Plot the results
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)
    plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
    forecast.plot(color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

# Evaluate metrics
evaluator = Evaluator()
agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
print(agg_metrics)


In [ ]:
file_path = 'dataset.xlsx'
data = pd.read_excel(file_path)

# Preprocess the data: Select only AREA columns and system column
df = data.copy()
df['ds'] = pd.to_datetime(df['ds'])
df.set_index('ds', inplace=True)

# Ensure 'system' column is present
if 'System' not in df.columns:
    raise ValueError("'System' column is not present in the dataset.")

# Check for missing values and fill them
df = df.fillna(df.mean())

# Standardize features except the target 'System'
scaler = StandardScaler()
features = df.drop(columns=['System'])
features_scaled = scaler.fit_transform(features)
df_scaled = pd.DataFrame(features_scaled, index=features.index, columns=features.columns)
df_scaled['System'] = df['System']

# Split the dataset into training, validation, and test sets
train_df, temp_df = train_test_split(df_scaled, test_size=0.3, shuffle=False)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=False)

# Create ListDataset for GluonTS
train_data = ListDataset(
    [{"start": train_df.index[0], "target": train_df['System'].values, "feat_static_cat": [0]}],
    freq="H"
)
validation_data = ListDataset(
    [{"start": validation_df.index[0], "target": validation_df['System'].values, "feat_static_cat": [0]}],
    freq="H"
)
test_data = ListDataset(
    [{"start": test_df.index[0], "target": test_df['System'].values, "feat_static_cat": [0]}],
    freq="H"
)

In [ ]:
def get_lag_llama_predictions(dataset, prediction_length, context_length=32, num_samples=20, device="cuda", batch_size=64, nonnegative_pred_samples=True):
    ckpt = torch.load("lag-llama.ckpt", map_location=device)
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        nonnegative_pred_samples=nonnegative_pred_samples,
        rope_scaling={
            "type": "linear",
            "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        },
        batch_size=batch_size,
        num_parallel_samples=num_samples,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(tqdm(forecast_it, total=len(dataset), desc="Forecasting batches"))
    tss = list(tqdm(ts_it, total=len(dataset), desc="Ground truth"))

    return forecasts, tss


In [ ]:
# Define parameters
prediction_length = 24
context_length = prediction_length * 3
num_samples = 20
device = "cuda"

# Fine-tune the model
ckpt = torch.load("lag-llama.ckpt", map_location=device)
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

estimator = LagLlamaEstimator(
    ckpt_path="lag-llama.ckpt",
    prediction_length=prediction_length,
    context_length=context_length,
    input_size=estimator_args["input_size"],
    n_layer=estimator_args["n_layer"],
    n_embd_per_head=estimator_args["n_embd_per_head"],
    n_head=estimator_args["n_head"],
    scaling=estimator_args["scaling"],
    time_feat=estimator_args["time_feat"],
    nonnegative_pred_samples=True,
    batch_size=32,  # Reduced batch size for better learning
    num_parallel_samples=num_samples,
    trainer_kwargs={"max_epochs": 100},  # Increased epochs
)

predictor = estimator.train(train_data, cache_data=True, shuffle_buffer_length=1000)


In [ ]:
# Evaluate the model
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data,
    predictor=predictor,
    num_samples=num_samples
)

forecasts = list(tqdm(forecast_it, total=len(test_data), desc="Forecasting batches"))
tss = list(tqdm(ts_it, total=len(test_data), desc="Ground truth"))

# Plot the results
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)
    plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
    forecast.plot(color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

# Evaluate metrics
evaluator = Evaluator()
agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
print(agg_metrics)
